# XGBoost ML Betting Strategy - Training Workflow

This notebook demonstrates the complete workflow for building an ML-based betting strategy using XGBoost.

## Overview

We'll build a binary classifier that predicts win probability for NBA games using:
- Historical odds data from multiple bookmakers
- Line movement patterns
- Sharp vs retail odds discrepancies
- Market consensus and efficiency metrics

The trained model integrates seamlessly with the backtesting framework via `BetOpportunity.confidence`.

## Prerequisites

Ensure dependencies are installed:
```bash
uv add xgboost scikit-learn numpy
```

In [ ]:
# Import required libraries
import asyncio
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBClassifier

# Project imports
from analytics.backtesting import BacktestConfig, BacktestEngine
from analytics.backtesting.models import BacktestEvent
from analytics.ml_strategy_example import FeatureEngineering, XGBoostStrategy
from core.config import Settings
from core.database import get_async_session
from core.models import EventStatus
from storage.readers import OddsReader

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Load settings
settings = Settings()

print("✓ Imports successful")

## 1. Data Preparation

Load historical events with final scores and their corresponding odds snapshots.

In [ ]:
async def load_training_data(
    start_date: datetime,
    end_date: datetime,
    decision_hours_before: int = 1
) -> tuple[list[BacktestEvent], list[list]]:
    """
    Load historical events and their odds snapshots for training.
    
    Args:
        start_date: Start of training period
        end_date: End of training period
        decision_hours_before: Hours before game to get odds (default: 1)
    
    Returns:
        Tuple of (events, odds_snapshots)
    """
    async with get_async_session() as session:
        reader = OddsReader(session)
        
        # Get completed events with scores
        events = await reader.get_events_by_date_range(
            start_date=start_date,
            end_date=end_date,
            status=EventStatus.FINAL
        )
        
        print(f"Found {len(events)} completed events")
        
        # Convert to BacktestEvent and get odds
        backtest_events = []
        odds_snapshots = []
        
        for event in events:
            bt_event = BacktestEvent.from_db_event(event)
            if bt_event is None:
                continue
            
            # Get odds at decision time
            decision_time = event.commence_time - timedelta(hours=decision_hours_before)
            odds = await reader.get_odds_at_time(
                event.id,
                decision_time,
                tolerance_minutes=30
            )
            
            if odds:
                backtest_events.append(bt_event)
                odds_snapshots.append(odds)
        
        print(f"Loaded {len(backtest_events)} events with odds data")
        
        return backtest_events, odds_snapshots

# Load data for training period (adjust dates based on your data)
TRAIN_START = datetime(2024, 10, 1)
TRAIN_END = datetime(2024, 11, 30)

events, odds_snapshots = await load_training_data(TRAIN_START, TRAIN_END)

print(f"\nTraining data: {len(events)} games from {TRAIN_START.date()} to {TRAIN_END.date()}")

## 2. Feature Engineering

Extract features from each event and create training dataset.

In [ ]:
def create_training_dataset(
    events: list[BacktestEvent],
    odds_snapshots: list[list],
    market: str = "h2h"
) -> tuple[pd.DataFrame, np.ndarray]:
    """
    Create training dataset from events and odds.
    
    For each game, we create two samples:
    - Home team sample (label=1 if home won, 0 otherwise)
    - Away team sample (label=1 if away won, 0 otherwise)
    
    Args:
        events: List of completed events
        odds_snapshots: Corresponding odds snapshots
        market: Market to analyze (h2h, spreads, totals)
    
    Returns:
        Tuple of (features_df, labels_array)
    """
    samples = []
    labels = []
    
    for event, odds in zip(events, odds_snapshots):
        # Home team sample
        home_features = FeatureEngineering.extract_features(
            event, odds, market=market, outcome=event.home_team
        )
        
        if home_features:
            samples.append(home_features)
            # Label: 1 if home won, 0 otherwise
            labels.append(1 if event.home_score > event.away_score else 0)
        
        # Away team sample
        away_features = FeatureEngineering.extract_features(
            event, odds, market=market, outcome=event.away_team
        )
        
        if away_features:
            samples.append(away_features)
            # Label: 1 if away won, 0 otherwise
            labels.append(1 if event.away_score > event.home_score else 0)
    
    # Convert to DataFrame and array
    features_df = pd.DataFrame(samples)
    labels_array = np.array(labels)
    
    return features_df, labels_array

# Create training dataset
features_df, labels = create_training_dataset(events, odds_snapshots)

print(f"Dataset shape: {features_df.shape}")
print(f"Class distribution: {np.bincount(labels)}")
print(f"Win rate: {labels.mean():.2%}\n")

# Display sample of features
print("Feature names:")
for i, col in enumerate(features_df.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nFirst sample features:")
features_df.head()

## 3. Train/Test Split

Split data for training and evaluation.

In [ ]:
# Split data (80/20 train/test)
X_train, X_test, y_train, y_test = train_test_split(
    features_df.values,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining win rate: {y_train.mean():.2%}")
print(f"Test win rate: {y_test.mean():.2%}")

## 4. Model Training

Train XGBoost classifier with default parameters first.

In [ ]:
# Train initial model with default parameters
strategy = XGBoostStrategy()

strategy.train(
    X_train=X_train,
    y_train=y_train,
    feature_names=list(features_df.columns),
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

print("✓ Model trained successfully")

## 5. Model Evaluation

Evaluate model performance on test set.

In [ ]:
# Make predictions
y_pred = strategy.model.predict(X_test)
y_pred_proba = strategy.model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Model Performance:")
print(f"  Accuracy: {accuracy:.2%}")
print(f"  ROC-AUC: {roc_auc:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Loss", "Win"]))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

### ROC Curve

In [ ]:
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, linewidth=2, label=f"XGBoost (AUC = {roc_auc:.4f})")
plt.plot([0, 1], [0, 1], "k--", linewidth=1, label="Random Classifier")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Calibration Plot

In [ ]:
# Plot calibration (predicted probability vs actual win rate)
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)

plt.figure(figsize=(10, 6))
plt.plot(prob_pred, prob_true, marker="o", linewidth=2, label="XGBoost")
plt.plot([0, 1], [0, 1], "k--", linewidth=1, label="Perfect Calibration")
plt.xlabel("Predicted Probability")
plt.ylabel("Actual Win Rate")
plt.title("Calibration Plot (Reliability Curve)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Note: A well-calibrated model has predictions close to the diagonal line.")

## 6. Feature Importance

Analyze which features are most important for predictions.

In [ ]:
# Get feature importance
feature_importance = strategy.get_feature_importance()

# Sort by importance
importance_df = pd.DataFrame({
    "Feature": list(feature_importance.keys()),
    "Importance": list(feature_importance.values())
}).sort_values("Importance", ascending=False)

# Plot top 15 features
plt.figure(figsize=(12, 8))
top_features = importance_df.head(15)
plt.barh(range(len(top_features)), top_features["Importance"])
plt.yticks(range(len(top_features)), top_features["Feature"])
plt.xlabel("Importance Score")
plt.title("Top 15 Most Important Features")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 Features:")
print(importance_df.head(10).to_string(index=False))

## 7. Hyperparameter Tuning (Optional)

Use GridSearchCV to find optimal hyperparameters.

In [ ]:
# Define hyperparameter grid
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [4, 6, 8],
    "learning_rate": [0.05, 0.1, 0.2],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

# Grid search with 3-fold cross-validation
xgb = XGBClassifier(random_state=42, eval_metric="logloss")

grid_search = GridSearchCV(
    xgb,
    param_grid,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)

print("Running grid search (this may take several minutes)...")
grid_search.fit(X_train, y_train)

print("\n✓ Grid search complete")
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV ROC-AUC: {grid_search.best_score_:.4f}")

# Evaluate best model on test set
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]

accuracy_best = accuracy_score(y_test, y_pred_best)
roc_auc_best = roc_auc_score(y_test, y_pred_proba_best)

print(f"\nBest Model Test Performance:")
print(f"  Accuracy: {accuracy_best:.2%}")
print(f"  ROC-AUC: {roc_auc_best:.4f}")

# Update strategy with best model
strategy.model = best_model
print("\n✓ Strategy updated with best model")

## 8. Save Model

Save the trained model for use in backtesting and production.

In [ ]:
# Save model
model_path = "models/xgboost_h2h_v1.pkl"
strategy.save_model(model_path)

print(f"✓ Model saved to {model_path}")
print(f"\nModel details:")
print(f"  Features: {len(strategy.feature_names)}")
print(f"  Training samples: {X_train.shape[0]}")
print(f"  Test ROC-AUC: {roc_auc_best:.4f}")

## 9. Backtest the Strategy

Test the trained model on a separate time period using the backtesting framework.

In [ ]:
# Define backtest period (should be different from training period)
BACKTEST_START = datetime(2024, 12, 1)
BACKTEST_END = datetime(2024, 12, 31)

# Load the saved model
backtest_strategy = XGBoostStrategy(
    model_path=model_path,
    min_edge_threshold=0.03,  # Require 3% edge
    min_confidence=0.52       # Only bet if model predicts >52% win probability
)

# Configure backtest
config = BacktestConfig(
    start_date=BACKTEST_START,
    end_date=BACKTEST_END,
    initial_bankroll=10000.0,
    bet_sizing_method="fractional_kelly",
    kelly_fraction=0.25,  # Quarter-Kelly for safety
    max_bet_percentage=0.05,
    min_bet_size=10.0,
    max_bet_size=500.0,
    decision_hours_before_game=1
)

print(f"Running backtest from {BACKTEST_START.date()} to {BACKTEST_END.date()}...")

# Run backtest
async with get_async_session() as session:
    reader = OddsReader(session)
    engine = BacktestEngine(backtest_strategy, config, reader)
    result = await engine.run()

print("\n✓ Backtest complete")
print(result.to_summary_text())

### Analyze Results

In [ ]:
# Save results
result.to_json("backtest_results_xgboost.json")
result.to_csv("backtest_bets_xgboost.csv")

print("✓ Results saved")
print(f"  JSON: backtest_results_xgboost.json")
print(f"  CSV: backtest_bets_xgboost.csv")

# Plot equity curve
equity_df = pd.DataFrame([
    {"date": point.date, "bankroll": point.bankroll}
    for point in result.equity_curve
])

plt.figure(figsize=(14, 6))
plt.plot(equity_df["date"], equity_df["bankroll"], linewidth=2)
plt.axhline(y=config.initial_bankroll, color="r", linestyle="--", alpha=0.5, label="Starting Bankroll")
plt.xlabel("Date")
plt.ylabel("Bankroll ($)")
plt.title("XGBoost Strategy - Equity Curve")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nKey Metrics:")
print(f"  Total Bets: {result.total_bets}")
print(f"  Win Rate: {result.win_rate:.2f}%")
print(f"  ROI: {result.roi:.2f}%")
print(f"  Total Profit: ${result.total_profit:,.2f}")
print(f"  Sharpe Ratio: {result.sharpe_ratio:.2f}")
print(f"  Max Drawdown: ${abs(result.max_drawdown):,.2f} ({result.max_drawdown_percentage:.2f}%)")

## 10. Summary

This notebook demonstrated:

1. **Data Preparation**: Loading historical events with odds snapshots
2. **Feature Engineering**: Extracting 15+ features capturing market dynamics
3. **Model Training**: XGBoost classifier with hyperparameter tuning
4. **Evaluation**: ROC-AUC, calibration, feature importance analysis
5. **Backtesting**: Seamless integration with backtesting framework

### Next Steps

- **Improve Features**: Add more sophisticated features (e.g., team stats, injuries)
- **Feature Selection**: Use recursive feature elimination to reduce overfitting
- **Model Ensembles**: Combine XGBoost with other models (LightGBM, Neural Networks)
- **Time-Series Validation**: Use walk-forward analysis instead of random split
- **Spread/Totals Models**: Train separate models for spread and totals markets
- **Production Deployment**: Integrate with live odds fetching for real-time predictions

### Using This Pattern for Other Models

This workflow can be adapted for:
- **LightGBM**: Similar to XGBoost but potentially faster
- **Neural Networks**: Use PyTorch/TensorFlow for deep learning
- **Time Series Models**: LSTM/GRU for sequential line movement patterns
- **Ensemble Methods**: Stack multiple models for better predictions

See `analytics/ml_strategy_example.py` for implementation details.